In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.models import load_model, Model
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.activations import relu
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications import InceptionResNetV2


!pip install kaggle
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d sujaykapadnis/monkeypox-skin-images-dataset-msid
!unzip -q monkeypox-skin-images-dataset-msid


 79% 43.0M/54.6M [00:00<00:00, 99.7MB/s]
100% 54.6M/54.6M [00:00<00:00, 102MB/s] 


In [3]:
dataPath = '/content/Monkeypox Skin Images Dataset (MSID)/Monkeypox Skin Image Dataset/Monkeypox Skin Image Dataset'
img_height = 224
img_width = 224
dataGen = ImageDataGenerator(
    rescale=1/255.,
    validation_split = 0.2
)
TrainData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='training',
)
validationData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='validation',
)


baseModel = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=((img_height, img_width, 3)))

for layer in baseModel.layers:
  layer.trainable = False

input_layer = Input(shape=(224,224,3))
modelOuput = baseModel(input_layer) # Pass the input through the VGG16 base model
fOutput = Flatten()(modelOuput) # Flatten the output and add additional fully connected layers
dense_1 = Dense(512, activation='relu')(fOutput)
dense2 = Dense(256, activation='relu')(dense_1)
output_layer = Dense(4, activation='softmax')(dense2)

model = Model(inputs=input_layer, outputs=output_layer)
# Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
history = model.fit(
    TrainData,
    epochs=50,  # adjust the number of epochs as needed
    validation_data=validationData
)

Found 618 images belonging to 4 classes.
Found 152 images belonging to 4 classes.
219055592/219055592 [==============================] - 2s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 inception_resnet_v2 (Funct  (None, 5, 5, 1536)        54336736  
 ional)                                                          
                                                                 
 flatten (Flatten)           (None, 38400)             0         
                                                                 
 dense (Dense)               (None, 512)               19661312  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                            